In [8]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

# Cargar datos
df = pd.read_csv("profesores_uc3m_v3.csv")

print("=== DATOS ORIGINALES ===")
print(f"Total filas: {len(df)}")
print(f"Columnas: {df.columns.tolist()}")
print("\nPrimeras filas:")
print(df.head(10))
print("\nValores únicos en departamento:")
print(df["departamento"].value_counts().head(10))

=== DATOS ORIGINALES ===
Total filas: 4199
Columnas: ['departamento', 'profesor', 'email']

Primeras filas:
    departamento                        profesor  \
0  1371206569261                    Departamento   
1  1371206569261                             NaN   
2  1371206569261                             NaN   
3  1371206569261                             NaN   
4  1371206569261                             NaN   
5  1371206569261                             NaN   
6  1371206569261  alameda castillo, maria teresa   
7  1371206569261  alvarado caycho, gisella rocio   
8  1371206569261            anton juarez, isabel   
9  1371206569261          areal ludeña, santiago   

                           email  
0           maplazaa@pa.uc3m.es.  
1           maplazaa@pa.uc3m.es.  
2      maplazaa@pa.uc3m.es.Ficha  
3                    dpd@uc3m.es  
4            maplazaa@pa.uc3m.es  
5       maplazaa@pa.uc3m.esBasic  
6    mariateresa.alameda@uc3m.es  
7  gisellarocio.alvarado@uc3m.es  
8   

In [9]:
# Estadísticas generales 
total_filas = len(df)
correos_unicos = df["email"].nunique()
profesores_con_nombre = df["profesor"].notna().sum()
profesores_sin_nombre = df["profesor"].isna().sum()
departamentos_unicos = df["departamento"].nunique()

print("=== ESTADÍSTICAS INICIALES ===")
print(f"Total filas en el CSV: {total_filas}")
print(f"Correos únicos: {correos_unicos}")
print(f"Profesores con nombre identificado: {profesores_con_nombre}")
print(f"Profesores sin nombre: {profesores_sin_nombre}")
print(f"Número de departamentos únicos: {departamentos_unicos}")
print(f"\nEjemplos de emails problemáticos:")
print(df[df["email"].str.contains(r"\.(Ficha|Basic|alvarado|anton)", na=False, regex=True)]["email"].head(10).tolist())

=== ESTADÍSTICAS INICIALES ===
Total filas en el CSV: 4199
Correos únicos: 3793
Profesores con nombre identificado: 1919
Profesores sin nombre: 2280
Número de departamentos únicos: 1

Ejemplos de emails problemáticos:
['maplazaa@pa.uc3m.es.Ficha', 'gisellarocio.alvarado@uc3m.es', 'isabel.anton@uc3m.es', 'rociogisellarocio.alvarado@uc3m.esanton', 'isabelisabel.anton@uc3m.esareal', 'isabel.anton@uc3m.es', 'gisellarocio.alvarado@uc3m.es', 'rociogisellarocio.alvarado@uc3m.esbarreira', 'mariamar.antonino@uc3m.es', 'marmariamar.antonino@uc3m.escamblor']


/var/folders/89/t9vgkdv52f57k0g7jq3p9c4h0000gn/T/ipykernel_5077/2959259483.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(df[df["email"].str.contains(r"\.(Ficha|Basic|alvarado|anton)", na=False, regex=True)]["email"].head(10).tolist())


In [10]:
# limpiar emails
print("LIMPIEZA DE EMAILS")

# Regex robusta para emails
email_regex = r"[a-zA-Z0-9._%+-]+@(?:[a-zA-Z0-9.-]+\.)+[a-zA-Z]{2,}"

def extract_clean_email(text):
    """Extrae email limpio del texto, manejando casos con texto adicional"""
    if pd.isna(text):
        return None
    
    text = str(text).strip()
    
    # Buscar el primer email válido
    match = re.search(email_regex, text)
    if match:
        email = match.group(0)
        # Limpiar puntos finales y espacios
        email = email.rstrip('. ')
        return email.lower()
    return None

df["email_clean"] = df["email"].apply(extract_clean_email)

# Eliminar filas sin email válido
filas_antes = len(df)
df = df[df["email_clean"].notna()].copy()
filas_despues = len(df)
print(f"Filas eliminadas sin email válido: {filas_antes - filas_despues}")

# Verificar emails únicos
print(f"Emails únicos después de limpieza: {df['email_clean'].nunique()}")


=== LIMPIEZA DE EMAILS ===
Filas eliminadas sin email válido: 0
Emails únicos después de limpieza: 3789


In [11]:
# nombres

print("LIMPIEZA DE NOMBRES")

def clean_name(name):
    """Limpia y normaliza nombres de profesores"""
    if pd.isna(name) or str(name).strip() == "":
        return None
    
    name = str(name).strip()
    
    # Eliminar "Departamento" como nombre
    if name.lower() == "departamento":
        return None
    
    # Normalizar: quitar dobles espacios, normalizar mayúsculas
    name = re.sub(r"\s+", " ", name)
    # Capitalizar cada palabra (formato título)
    name = name.title()
    
    # Validar que tenga al menos 2 palabras y no sea demasiado largo
    words = name.split()
    if len(words) < 2 or len(words) > 8:
        return None
    
    return name

df["profesor_clean"] = df["profesor"].apply(clean_name)

print(f"Profesores con nombre después de limpieza: {df['profesor_clean'].notna().sum()}")
print(f"Profesores sin nombre: {df['profesor_clean'].isna().sum()}")


=== LIMPIEZA DE NOMBRES ===
Profesores con nombre después de limpieza: 1892
Profesores sin nombre: 2307


In [12]:
# MAPEO DE DEPARTAMENTOS (IDs → NOMBRES)


print("MAPEO DE DEPARTAMENTOS")

# Obtener lista de departamentos únicos con sus IDs
departamentos_ids = df["departamento"].unique()
print(f"Departamentos únicos encontrados: {len(departamentos_ids)}")

# Intentar cargar mapeo guardado previamente
import os
import json

MAPEO_FILE = "departamento_map.json"

def obtener_nombres_departamentos_desde_lista():
    """Obtiene todos los nombres de departamentos desde la página principal (más eficiente)"""
    BASE_URL = "https://www.uc3m.es"
    dept_list_url = f"{BASE_URL}/conocenos/departamentos"
    mapeo = {}
    
    try:
        response = requests.get(dept_list_url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a.get("href", "")
                if "/Detalle/Organismo_C/" in href:
                    # Extraer ID del departamento
                    match = re.search(r'/Detalle/Organismo_C/(\d+)', href)
                    if match:
                        dep_id = match.group(1)
                        nombre = a.get_text(strip=True)
                        if nombre and len(nombre) > 3:
                            mapeo[dep_id] = nombre
        return mapeo
    except Exception as e:
        print(f"Error obteniendo lista de departamentos: {e}")
        return {}

def obtener_nombre_departamento_individual(dep_id):
    """Obtiene el nombre de un departamento específico desde su página (fallback)"""
    BASE_URL = "https://www.uc3m.es"
    url = f"{BASE_URL}/conocenos/departamentos/Detalle/Organismo_C/{dep_id}"
    
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Buscar el título o nombre del departamento
            h1 = soup.find("h1")
            if h1:
                nombre = h1.get_text(strip=True)
                if nombre and len(nombre) > 3:
                    return nombre
            
            # Alternativa: buscar en meta tags o títulos
            title = soup.find("title")
            if title:
                nombre = title.get_text(strip=True)
                nombre = re.sub(r'\s*-\s*UC3M.*', '', nombre, flags=re.IGNORECASE)
                if nombre and len(nombre) > 3:
                    return nombre
    except Exception as e:
        pass  # Silenciar errores en fallback
    
    return None

# Cargar mapeo existente si existe
departamento_map = {}
if os.path.exists(MAPEO_FILE):
    with open(MAPEO_FILE, "r", encoding="utf-8") as f:
        departamento_map = json.load(f)
    print(f"Mapeo cargado desde {MAPEO_FILE} ({len(departamento_map)} departamentos)")

# Obtener nombres desde la página principal (más eficiente)
print("Obteniendo nombres de departamentos desde la página principal...")
mapeo_desde_lista = obtener_nombres_departamentos_desde_lista()
print(f"Encontrados {len(mapeo_desde_lista)} departamentos en la lista principal")

# Actualizar mapeo con los nombres obtenidos
departamento_map.update(mapeo_desde_lista)

# Completar mapeo para IDs que faltan
print("\nCompletando mapeo para departamentos faltantes...")
nuevos = 0
for dep_id in departamentos_ids:
    if str(dep_id) not in departamento_map:
        if str(dep_id).isdigit():
            # Intentar obtener desde página individual (fallback)
            nombre = obtener_nombre_departamento_individual(dep_id)
            if nombre:
                departamento_map[str(dep_id)] = nombre
                print(f"  {dep_id} → {nombre}")
                nuevos += 1
            else:
                departamento_map[str(dep_id)] = f"Departamento_{dep_id}"
            time.sleep(0.3)  # Pausa para no sobrecargar el servidor
        else:
            # Si ya es un nombre, mantenerlo
            departamento_map[str(dep_id)] = str(dep_id)

# Guardar mapeo para uso futuro
if nuevos > 0 or len(mapeo_desde_lista) > 0:
    with open(MAPEO_FILE, "w", encoding="utf-8") as f:
        json.dump(departamento_map, f, ensure_ascii=False, indent=2)
    print(f"\nMapeo guardado en {MAPEO_FILE}")

print(f"\nMapeo completado: {len(departamento_map)} departamentos")


=== MAPEO DE DEPARTAMENTOS ===
Departamentos únicos encontrados: 1
Obteniendo nombres de departamentos desde la página principal...
Encontrados 28 departamentos en la lista principal

Completando mapeo para departamentos faltantes...

Mapeo guardado en departamento_map.json

Mapeo completado: 29 departamentos


In [13]:
# Aplicar mapeo de departamentos
df["departamento_clean"] = df["departamento"].map(departamento_map).fillna(df["departamento"])

print("Departamentos después del mapeo:")
print(df["departamento_clean"].value_counts())


Departamentos después del mapeo:
departamento_clean
1371206569261    4199
Name: count, dtype: int64


/var/folders/89/t9vgkdv52f57k0g7jq3p9c4h0000gn/T/ipykernel_5077/2277881909.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["departamento_clean"] = df["departamento"].map(departamento_map).fillna(df["departamento"])


In [14]:
# INTENTAR INFERIR NOMBRES DESDE EMAILS

def infer_name_from_email(email):
    """Intenta inferir el nombre del profesor desde su email"""
    if pd.isna(email):
        return None
    
    # Patrón común: nombre.apellido@uc3m.es
    # Extraer la parte antes del @
    local_part = email.split("@")[0]
    
    # Dividir por puntos
    parts = local_part.split(".")
    
    if len(parts) >= 2:
        # Tomar las primeras partes como nombre y apellido
        # Normalmente: nombre.apellido o nombreapellido.apellido2
        name_parts = []
        for part in parts[:3]:  # Máximo 3 partes
            if part and len(part) > 2:
                name_parts.append(part.capitalize())
        
        if len(name_parts) >= 2:
            return " ".join(name_parts)
    
    return None

# Solo inferir nombres para registros que no tienen nombre
mask_sin_nombre = df["profesor_clean"].isna()
nombres_inferidos = df.loc[mask_sin_nombre, "email_clean"].apply(infer_name_from_email)

# Actualizar solo donde se pudo inferir
df.loc[mask_sin_nombre & nombres_inferidos.notna(), "profesor_clean"] = nombres_inferidos[mask_sin_nombre & nombres_inferidos.notna()]

print(f"Nombres inferidos desde emails: {nombres_inferidos.notna().sum()}")
print(f"Total profesores con nombre ahora: {df['profesor_clean'].notna().sum()}")


=== INFERENCIA DE NOMBRES DESDE EMAILS ===
Nombres inferidos desde emails: 2018
Total profesores con nombre ahora: 3910


In [15]:
# duplicados
print("=== ELIMINACIÓN DE DUPLICADOS ===")

filas_antes = len(df)
# Eliminar duplicados por email (mantener el primero que tenga nombre si es posible)
df = df.sort_values(by=["profesor_clean"], na_position="last").drop_duplicates(subset="email_clean", keep="first")
filas_despues = len(df)

print(f"Duplicados eliminados: {filas_antes - filas_despues}")
print(f"Filas finales: {filas_despues}")


=== ELIMINACIÓN DE DUPLICADOS ===
Duplicados eliminados: 410
Filas finales: 3789


In [16]:

# Reordenar columnas y renombrar
df_final = df[["departamento_clean", "profesor_clean", "email_clean"]].copy()
df_final.columns = ["departamento", "profesor", "email"]

# Ordenar por departamento y nombre
df_final = df_final.sort_values(by=["departamento", "profesor"]).reset_index(drop=True)

print("=== DATOS FINALES ===")
print(f"Total registros: {len(df_final)}")
print(f"Emails únicos: {df_final['email'].nunique()}")
print(f"Profesores con nombre: {df_final['profesor'].notna().sum()}")
print(f"Profesores sin nombre: {df_final['profesor'].isna().sum()}")
print(f"Departamentos únicos: {df_final['departamento'].nunique()}")

print("\nPrimeras filas:")
display(df_final.head(15))

print("\nDistribución por departamento:")
print(df_final["departamento"].value_counts())


=== DATOS FINALES ===
Total registros: 3789
Emails únicos: 3789
Profesores con nombre: 3603
Profesores sin nombre: 186
Departamentos únicos: 1

Primeras filas:


,departamento,profesor,email
0,1371206569261,06e-mailanamaria Blanes,06e-mailanamaria.blanes@uc3m.esweb
1,1371206569261,11e-mailmarianatalia Mato,11e-mailmarianatalia.mato@uc3m.escecilia.ferna...
2,1371206569261,12e-maildepartamento Hga,12e-maildepartamento.hga@hum.uc3m.essusanaa
3,1371206569261,12e-mailsecrethum Fll,12e-mailsecrethum.fll@hum.uc3m.es
4,1371206569261,30e-mailfrancisco Velasco,30e-mailfrancisco.velasco@uc3m.esweb
5,1371206569261,30e-mailmangeles Malfaz,30e-mailmangeles.malfaz@uc3m.esweb
6,1371206569261,32e-maildep Tyf,32e-maildep.tyf@uc3m.esweb
7,1371206569261,48e-maildepartamento Estadistica,48e-maildepartamento.estadistica@uc3m.esweb
8,1371206569261,49e-maildepartamento Informatica,49e-maildepartamento.informatica@uc3m.esweb
9,1371206569261,56e-mailmariajose Gutierrez,56e-mailmariajose.gutierrez@uc3m.esivan.barbeitos



Distribución por departamento:
departamento
1371206569261    3789
Name: count, dtype: int64


In [17]:

# Guardar CSV limpio
df_final.to_csv("profesores_uc3m_limpio.csv", index=False, encoding="utf-8")
print("✓ CSV limpio guardado como: profesores_uc3m_limpio.csv")

# Guardar también un CSV solo con profesores que tienen nombre
df_con_nombre = df_final[df_final["profesor"].notna()].copy()
df_con_nombre.to_csv("profesores_uc3m_con_nombres.csv", index=False, encoding="utf-8")
print(f"✓ CSV con nombres guardado como: profesores_uc3m_con_nombres.csv ({len(df_con_nombre)} registros)")

# Guardar estadísticas
stats = {
    "total_registros": len(df_final),
    "emails_unicos": df_final["email"].nunique(),
    "profesores_con_nombre": df_final["profesor"].notna().sum(),
    "profesores_sin_nombre": df_final["profesor"].isna().sum(),
    "departamentos_unicos": df_final["departamento"].nunique(),
}




✓ CSV limpio guardado como: profesores_uc3m_limpio.csv
✓ CSV con nombres guardado como: profesores_uc3m_con_nombres.csv (3603 registros)

=== ESTADÍSTICAS FINALES ===
total_registros: 3789
emails_unicos: 3789
profesores_con_nombre: 3603
profesores_sin_nombre: 186
departamentos_unicos: 1


In [18]:

# Emails que no son de uc3m.es
emails_no_uc3m = df_final[~df_final["email"].str.contains("@uc3m.es", na=False)]
if len(emails_no_uc3m) > 0:
    print(f"\nEmails que NO son de uc3m.es ({len(emails_no_uc3m)}):")
    print(emails_no_uc3m[["departamento", "profesor", "email"]].head(10))

# Departamentos con más profesores
print("\nTop 10 departamentos con más profesores:")
print(df_final["departamento"].value_counts().head(10))

# Departamentos con menos profesores
print("\nDepartamentos con menos profesores:")
print(df_final["departamento"].value_counts().tail(10))

# Porcentaje de cobertura de nombres por departamento
print("\nCobertura de nombres por departamento:")
cobertura = df_final.groupby("departamento").agg({
    "profesor": lambda x: (x.notna().sum() / len(x) * 100).round(1),
    "email": "count"
}).rename(columns={"profesor": "%_con_nombre", "email": "total"})
cobertura = cobertura.sort_values("%_con_nombre", ascending=False)
display(cobertura)


=== ANÁLISIS DE CALIDAD ===

Emails que NO son de uc3m.es (328):
      departamento                           profesor  \
2    1371206569261           12e-maildepartamento Hga   
3    1371206569261              12e-mailsecrethum Fll   
13   1371206569261                       91 624 57 67   
15   1371206569261                       91 624 57 69   
19   1371206569261                       91 624 87 17   
21   1371206569261                       91 624 93 01   
23   1371206569261                       91 624 95 30   
25   1371206569261                       91 624 96 98   
76   1371206569261  Agustín Redonodo Aparicio 11.0.09   
209  1371206569261        Alonso Perez Carlos Antonio   

                                           email  
2    12e-maildepartamento.hga@hum.uc3m.essusanaa  
3              12e-mailsecrethum.fll@hum.uc3m.es  
13                            tla@der-pr.uc3m.es  
15                          malba@der-pr.uc3m.es  
19                       pperales@der-pr.uc3m.es  
2

,%_con_nombre,total
departamento,,
1371206569261,95.1,3789
